In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import mapclassify as mc

Step0:  
Read Data and get basic county groups

0.0 Report Data

In [2]:
df_counties_original = pd.read_csv("./us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
#Tansform Unknown fips to '00000' 
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
df_counties_original.sort_values("fips", ascending=True)

,date,county,state,fips,cases,deaths
24767,2020-04-01,Unknown,Kentucky,00000,9,0
85713,2020-04-24,Unknown,Nevada,00000,43,5
48196,2020-04-10,Unknown,Utah,00000,0,7
48212,2020-04-10,Unknown,Vermont,00000,26,1
48216,2020-04-10,Unknown,Virgin Islands,00000,46,1
...,...,...,...,...,...,...
98259,2020-04-28,Washakie,Wyoming,56043,4,0
15835,2020-03-27,Washakie,Wyoming,56043,1,0
56536,2020-04-13,Washakie,Wyoming,56043,4,0
61966,2020-04-15,Washakie,Wyoming,56043,4,0


0.1 GeoJSON Data

In [3]:
old_counties_geojson_df = gpd.read_file(r"./counties_update_new.geojson")
old_counties_geojson_df.head(5)

,NAME,state_name,GEOID,population,geometry
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


Group Data to Counties

In [4]:
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties = pivot_counties.replace(np.nan, 0)
pivot_counties

cases                                   \
date                     2020-01-21 2020-01-22 2020-01-23 2020-01-24   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        0.0        0.0        0.0        0.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0   
        Washakie   56043        0.0        0.0        0.0        0.0   

                                                                      \
date                     2020-01-25 2020-01-26 2020-01-27 2020-01-28   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        0.0        0.0        0.0        0.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0   
        Washakie   56043        0.0        0.0        0.0        0.0   

                                                ...     deaths             \
date                     2020-01-29 2020-01-30  ... 2020-04-20 2020-04-21   
state   county     fips                         ...                         
Alabama Autauga    01001        0.0        0.0  ...        1.0        1.0   
        Baldwin    01003        0.0        0.0  ...        3.0        3.0   
        Barbour    01005        0.0        0.0  ...        0.0        0.0   
        Bibb       01007        0.0        0.0  ...        0.0        0.0   
        Blount     01009        0.0        0.0  ...        0.0        0.0   
...                             ...        ...  ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0  ...        0.0        0.0   
        Sweetwater 56037        0.0        0.0  ...        0.0        0.0   
        Teton      56039        0.0        0.0  ...        0.0        0.0   
        Uinta      56041        0.0        0.0  ...        0.0        0.0   
        Washakie   56043        0.0        0.0  ...        0.0        0.0   

                                                                      \
date                     2020-04-22 2020-04-23 2020-04-24 2020-04-25   
state   county     fips                                                
Alabama Autauga    01001        2.0        2.0        2.0        2.0   
        Baldwin    01003        3.0        3.0        3.0        3.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        1.0        1.0        1.0   
        Uinta      56041        0.0        0.0        0.0        0.0 

0.3 Get Date Info

In [5]:
date= pivot_counties['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
       '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
       '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19',
       '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
       '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
       '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10',
       '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15',
       '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20',
       '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',

In [6]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [7]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

100 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01',
               '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05',
               '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
               '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13',
               '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
               '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21',
               '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25',
               '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
               '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16',
  

Step1:  
Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [8]:
# import json
# pivot_counties['cases_ts'] = json.dumps({"values": pivot_counties['cases'].values.tolist()[0]})
# pivot_counties['deaths_ts'] =  json.dumps({"values": pivot_counties['deaths'].values.tolist()[0]})

pivot_counties['cases_ts'] =  pivot_counties['cases'].values.tolist()
pivot_counties['deaths_ts'] =  pivot_counties['deaths'].values.tolist()

Form Dynamic classes for new_case and new_death 

In [9]:
new_case_dict = {}
for i in range(1,len(date)):
    q6 = mc.Quantiles(pivot_counties['cases_ts'].apply(lambda x: x[i] - x[i - 1]).values, k=6)
    new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

/Users/zhiyul/miniconda3/envs/wherecovid19/lib/python3.8/site-packages/mapclassify/classifiers.py:236: UserWarning: Warning: Not enough unique values in array to form k classes
  Warn(
/Users/zhiyul/miniconda3/envs/wherecovid19/lib/python3.8/site-packages/mapclassify/classifiers.py:239: UserWarning: Warning: setting k to 1
  Warn("Warning: setting k to %d" % k_q, UserWarning)
/Users/zhiyul/miniconda3/envs/wherecovid19/lib/python3.8/site-packages/mapclassify/classifiers.py:890: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/Users/zhiyul/miniconda3/envs/wherecovid19/lib/python3.8/site-packages/mapclassify/classifiers.py:239: UserWarning: Warning: setting k to 2
  Warn("Warning: setting k to %d" % k_q, UserWarning)
/Users/zhiyul/miniconda3/envs/wherecovid19/lib/python3.8/site-packages/mapclassify/classifiers.py:239: UserWarning: Warning: setting k to 3
  Warn("Warning: setting k to %d" % k_q, UserWarning)
/Users/zhiyul/miniconda3/envs/wherecovid

In [10]:
dynamic_cl_dict = {}
dynamic_cl_dict['county'] = {'new_case':new_case_dict, 'new_death':{}}
dynamic_cl_dict['state'] = {}

In [11]:
import json
with open('dynamic_classes.json','w') as json_file:
    json.dump(dynamic_cl_dict, json_file)

In [12]:
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths counties
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties.head(5)

cases                                              \
date                  2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                                                                              \
date                  2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                       ...     deaths                                   \
date                   ... 2020-04-24 2020-04-25 2020-04-26 2020-04-27   
state   county  fips   ...                                               
Alabama Autauga 01001  ...        2.0        2.0        2.0        3.0   
        Baldwin 01003  ...        3.0        3.0        3.0        3.0   
        Barbour 01005  ...        0.0        0.0        0.0        0.0   
        Bibb    01007  ...        0.0        0.0        0.0        0.0   
        Blount  01009  ...        0.0        0.0        0.0        0.0   

                                             \
date                  2020-04-28 2020-04-29   
state   county  fips                          
Alabama Autauga 01001        4.0        4.0   
        Baldwin 01003        3.0        2.0   
        Barbour 01005        0.0        1.0   
        Bibb    01007        0.0        0.0   
        Blount  01009        0.0        0.0   

                                                                cases_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                               deaths_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                      dt_first_case dt_first_death  
date                                                
state   county  fips                                
Alabama Autauga 01001    2020-03-24     2020-04-06  
        Baldwin 01003    2020-03-14     2020-04-01  
        Barbour 01005    2020-04-03     2020-04-29  
        Bibb    01007    2020-03-30            NaN  
        Blount  01009    2020-03-25            NaN  

[5 rows x 204 columns]

Step2:  
Add today_case, today_new_case, today_death, today_new_death

In [13]:
yesterday = date.values[-2]
pivot_counties['today_case'] = pivot_counties['cases'][dt_str_end]
pivot_counties['today_new_case'] = pivot_counties['cases'][dt_str_end] - pivot_counties['cases'][yesterday]
pivot_counties['today_death'] = pivot_counties['deaths'][dt_str_end]
pivot_counties['today_new_death'] = pivot_counties['deaths'][dt_str_end] - pivot_counties['deaths'][yesterday]

Step3:  
Extract necessary columns

In [14]:
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,Autauga,01001,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-24,2020-04-06,43.0,3.0,4.0,0.0
1,Alabama,Baldwin,01003,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-14,2020-04-01,173.0,2.0,2.0,-1.0
2,Alabama,Barbour,01005,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-04-03,2020-04-29,37.0,0.0,1.0,1.0
3,Alabama,Bibb,01007,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-30,NaN,42.0,0.0,0.0,0.0
4,Alabama,Blount,01009,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-25,NaN,36.0,2.0,0.0,0.0


In [15]:
report_df['cases_ts'][1]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 3.0,
 4.0,
 4.0,
 5.0,
 5.0,
 10.0,
 15.0,
 18.0,
 19.0,
 23.0,
 25.0,
 28.0,
 29.0,
 34.0,
 38.0,
 42.0,
 49.0,
 59.0,
 59.0,
 66.0,
 71.0,
 78.0,
 87.0,
 98.0,
 102.0,
 103.0,
 109.0,
 114.0,
 117.0,
 123.0,
 132.0,
 143.0,
 147.0,
 154.0,
 161.0,
 168.0,
 171.0,
 173.0]

Step4:  
Assign Geometries to counties

In [16]:
final_df = pd.merge(old_counties_geojson_df[["NAME", "state_name", "GEOID", "population", "geometry"]], report_df, how='left', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Index(['NAME', 'state_name', 'GEOID', 'population', 'geometry', 'state',
       'county', 'fips', 'cases_ts', 'deaths_ts', 'dt_first_case',
       'dt_first_death', 'today_case', 'today_new_case', 'today_death',
       'today_new_death'],
      dtype='object')

Step5:  
Deal with nan values (Counties not shows up in geojson) (More works)

In [17]:
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)
final_df['fips'] = final_df['fips'].replace(np.nan, final_df['GEOID'])


In [18]:
template =final_df[~final_df['cases_ts'].isna()]['cases_ts']
template = template.iloc[0]

In [19]:
for x in range(0,len(template)):
    template[x] = 0
template = ','.join(map(str, template))

In [20]:

final_df['cases_ts'] = final_df['cases_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df['deaths_ts'] = final_df['deaths_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df.head(5)

,NAME,state_name,GEOID,population,geometry,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",North Carolina,Bladen,37017,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-29,NaN,20.0,7.0,0.0,0.0
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",North Carolina,Stanly,37167,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,2020-04-09,28.0,0.0,4.0,0.0
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",Ohio,Summit,39153,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-14,2020-03-27,584.0,42.0,49.0,6.0
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",Pennsylvania,Sullivan,42113,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-04,NaN,1.0,0.0,0.0,0.0
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",Texas,Upshur,48459,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,NaN,13.0,0.0,0.0,0.0


Step6:  
Finalize the Dataframe

In [21]:
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 
                   'today_case','today_new_case','today_death','today_new_death', "geometry"]]
final_df['dt_start'] = dt_str_start
final_df['dt_end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(1)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,dt_start,dt_end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-03-29,NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",20.0,7.0,0.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-04-29,day


Step 7:  
Output file

In [22]:
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


In [23]:
illinois_gdf = final_df[final_df['state_name'] == 'Illinois']

In [24]:
illinois_gdf.to_file(r"./illinois/nyt_illinois_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print('done')

done


New Data Structure (Full version)

In [25]:
# pivot_counties.columns = ['_'.join(col).strip() for col in pivot_counties.columns.values]
# report_df_2 = pivot_counties.reset_index()

In [26]:
# report_df_2.head(5)

In [27]:
# pivot_counties.head(5)

In [28]:
# final_df_2 = pd.merge(old_counties_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df_2, how='right', left_on=['NAME','state_name'], right_on = ['county','state'])
# final_df_2 = final_df_2.drop(['state','county'], axis = 1)
# final_df_2['dt_start'] = dt_start
# final_df_2['dt_end'] = dt_end
# final_df_2.head(5)

In [29]:
# final_df_2.to_file(r"./nyt_counties_data_full.geojson", driver='GeoJSON', encoding='utf-8')
# print("done")

Generate JSON file

In [30]:
# county_list = []
# for state in df_counties_original.state.unique():
#     for county in df_counties_original.loc[(df_counties_original['state'] == state)].county.unique():
#         county_list.append({"name": county, "state": state})
 
# print(len(county_list), county_list)


In [31]:
# data_df = report_df.set_index(report_df["county"]+", "+report_df["state"])
# data_dict =data_df.to_dict(orient='index')

In [32]:
# df = df_counties_original.groupby(['county', 'state'], sort=False)[['cases', 'deaths']].max().sort_values("cases", ascending=False)
# county_state_list = list(map(lambda x: x[0]+", "+x[1], df.index.tolist()))
# meta = {"dates": dt_range_str,"states": county_state_list, "cases": df["cases"].values.tolist(), "deaths": df["deaths"].values.tolist()}
# data_dict["metadata"] = meta

In [33]:
# data_dict

In [34]:
# import json
# with open('./nyt_counties_data.json', 'w') as outfile:
#     json.dump(data_dict, outfile)